In [1]:
import yaml
import os

with open('data.yaml', 'r') as file:
    data = yaml.safe_load(file)

print("Train path:", os.path.exists(data['train']))
print("Validation path:", os.path.exists(data['val']))
print("Test path:", os.path.exists(data['test']))

def count_images(directory):
    supported_formats = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif')
    return len([name for name in os.listdir(directory) if name.lower().endswith(supported_formats)])

print(f" The number of images in training set: {count_images(data['train'])}")
print(f" The number of images in validation set: {count_images(data['val'])}")
print(f" The number of images in test set: {count_images(data['test'])}")


Train path: True
Validation path: True
Test path: True
 The number of images in training set: 1228
 The number of images in validation set: 168
 The number of images in test set: 84


In [2]:
from ultralytics import YOLO
from pathlib import Path

absolute_from_relative_path = Path('data.yaml').resolve()

model = YOLO('yolov8n.pt')  # Load a pretrained model (recommended for training)
results = model.train(data=absolute_from_relative_path, epochs=10, imgsz=480)


  0%|          | 0.00/6.23M [00:00<?, ?B/s]

New https://pypi.org/project/ultralytics/8.2.25 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.227 🚀 Python-3.9.18 torch-1.13.1 CPU (Apple M1)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/alessiacolumban/Desktop/2023-2024-projectone-ctai-AlessCol7/PillPoint/AI/PillPoint.v1i.yolov8/data.yaml, epochs=10, patience=50, batch=16, imgsz=480, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, au

Fontconfig warning: ignoring UTF-8: not a valid region tag
[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


Model summary: 225 layers, 3011433 parameters, 3011417 gradients

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /Users/alessiacolumban/Desktop/2023-2024-projectone-ctai-AlessCol7/PillPoint/AI/PillPoint.v1i.yolov8/train/labels... 1228 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1228/1228 [00:00<00:00, 2626.75it/s]

...

      10/10         0G      1.155     0.7848      1.193         28        480: 100%|██████████| 77/77 [05:13<00:00,  4.07s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:16<00:00,  2.71s/it]

                   all        168        422      0.922      0.917      0.957      0.629



10 epochs completed in 0.886 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.0.227 🚀 Python-3.9.18 torch-1.13.1 CPU (Apple M1)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:16<00:00,  2.68s/it]


                   all        168        422      0.922      0.917      0.957      0.629
                 Hands        168        261       0.86      0.854      0.917      0.494
                  Pill        168         98      0.942      0.959      0.978      0.718
               PillBox        168         63      0.964      0.937      0.975      0.675
Speed: 1.1ms preprocess, 88.9ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train


In [1]:
import cv2
from ultralytics import YOLO
from pathlib import Path

# Load the custom YOLOv8 model
model = YOLO('runs/detect/train/weights/best.pt')

# Try different indices to ensure the correct webcam is used
for index in range(3):
    cap = cv2.VideoCapture(index)  # or use cv2.VideoCapture(index, cv2.CAP_V4L2)

    if cap.isOpened():
        print(f"Successfully opened webcam with index {index}")
        break
else:
    print("Error: Could not open any webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Perform inference
    results = model(frame)

    # Display the results on the frame
    # for result in results:
    #     for box in result.boxes:
    #         # Ensure the coordinates are extracted correctly
    #         coords = box.xyxy[0].tolist()
    #         if len(coords) == 4:
    #             x1, y1, x2, y2 = map(int, coords)
    #             conf = box.conf[0]
    #             cls = box.cls[0]
    #             label = f'{model.names[int(cls)]} {conf:.2f}'
    #             # Draw the bounding box and label on the frame
    #             cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    #             cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    #         else:
    #             print("Unexpected coordinates format:", coords)
    # After inference
for result in results:
    for box in result.boxes:
        coords = box.xyxy[0].tolist()
        if len(coords) == 4:
            x1, y1, x2, y2 = map(int, coords)
            conf = box.conf[0]
            cls = box.cls[0]
            class_name = model.names[int(cls)]  # Get the class name
            label = f'{class_name} {conf:.2f}'
            if class_name in ['Pillbox', 'Hand', 'Pill']:  # Filter by class names
                # Draw the bounding box and label on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        else:
            print("Unexpected coordinates format:", coords)


    # Display the frame
    cv2.imshow('Webcam Detection', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


[ WARN:0@1.619] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Successfully opened webcam with index 0

0: 288x480 1 PillBox, 66.8ms
Speed: 14.2ms preprocess, 66.8ms inference, 13.1ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 PillBox, 46.1ms
Speed: 1.5ms preprocess, 46.1ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 480)



[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


0: 288x480 1 PillBox, 60.7ms
Speed: 1.9ms preprocess, 60.7ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 PillBox, 55.6ms
Speed: 1.4ms preprocess, 55.6ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 PillBox, 48.8ms
Speed: 1.1ms preprocess, 48.8ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 PillBox, 46.9ms
Speed: 1.3ms preprocess, 46.9ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 PillBox, 90.6ms
Speed: 1.6ms preprocess, 90.6ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 PillBox, 55.4ms
Speed: 1.5ms preprocess, 55.4ms inference, 0.6ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 PillBox, 65.3ms
Speed: 1.3ms preprocess, 65.3ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 PillBox, 113.4ms
Speed: 1.3ms preprocess, 113.4ms inference, 0.5ms postprocess per image at shape 

KeyboardInterrupt: 

In [2]:
import cv2
from ultralytics import YOLO
from pathlib import Path

# Load the custom YOLOv8 model
model = YOLO('runs/detect/train/weights/best.pt')

# Try different indices to ensure the correct webcam is used
for index in range(3):
    cap = cv2.VideoCapture(index)  # or use cv2.VideoCapture(index, cv2.CAP_V4L2)

    if cap.isOpened():
        print(f"Successfully opened webcam with index {index}")
        break
else:
    print("Error: Could not open any webcam.")
    exit()

identification_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Perform inference
    results = model(frame)

    # Display the results on the frame
    for result in results:
        for box in result.boxes:
            # Ensure the coordinates are extracted correctly
            coords = box.xyxy[0].tolist()
            if len(coords) == 4:
                x1, y1, x2, y2 = map(int, coords)
                conf = box.conf[0]
                cls = box.cls[0]
                class_name = model.names[int(cls)]  # Get the class name
                label = f'{class_name} {conf:.2f}'
                # Draw the bounding box and label on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                
                # Increment identification count
                identification_count += 1
                if identification_count >= 10:  # Limit to 10 identifications
                    break  # Exit the loop if the limit is reached
            else:
                print("Unexpected coordinates format:", coords)

    # Display the frame
    cv2.imshow('Webcam Detection', frame)

    # Break the loop on 'q' key press or after reaching the identification limit
    if cv2.waitKey(1) & 0xFF == ord('q') or identification_count >= 10:
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


[ WARN:0@148.554] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Successfully opened webcam with index 0

0: 288x480 1 PillBox, 70.8ms
Speed: 7.6ms preprocess, 70.8ms inference, 13.1ms postprocess per image at shape (1, 3, 288, 480)


In [ ]:
# import cv2
# from ultralytics import YOLO
# from pathlib import Path

# # Load the pretrained YOLOv8 model
# model = YOLO('yolov8n.pt')  # Replace with the path to your model if needed

# # Connect to the webcam
# cap = cv2.VideoCapture(0)

# if not cap.isOpened():
#     print("Error: Could not open webcam.")
#     exit()

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         print("Failed to grab frame")
#         break

#     # Perform inference
#     results = model(frame)

#     # Display the results on the frame
#     for result in results:
#         for box in result.boxes:
#             x1, y1, x2, y2 = map(int, box.xyxy)
#             conf = box.conf[0]
#             cls = box.cls[0]
#             label = f'{model.names[int(cls)]} {conf:.2f}'
#             # Draw the bounding box and label on the frame
#             cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#             cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

#     # Display the frame
#     cv2.imshow('Webcam Detection', frame)

#     # Break the loop on 'q' key press
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the webcam and close windows
# cap.release()
# cv2.destroyAllWindows()


In [ ]:
# import cv2
# import matplotlib.pyplot as plt
# import ipywidgets as widgets
# from IPython.display import display
# from ultralytics import YOLO
# import time

# # Load the trained model
# model = YOLO('runs/detect/train1/weights/best.pt')

# # Open a connection to the webcam (0 is the default camera)
# cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# # Check if the webcam is opened correctly
# if not cap.isOpened():
#     print("Error: Could not open webcam.")
#     exit()

# # Set up the output widget
# image_widget = widgets.Image(format='jpeg')
# display(image_widget)

# def capture_and_display():
#     while True:
#         # Capture frame-by-frame
#         ret, frame = cap.read()
#         if not ret:
#             print("Error: Could not read frame.")
#             break

#         # Resize frame if needed (optional)
#         img_size = 640
#         frame_resized = cv2.resize(frame, (img_size, img_size))

#         # Evaluate the model on the current frame
#         results = model.predict(source=frame_resized)

#         # Draw results on the frame
#         for result in results:
#             for i, box in enumerate(result.boxes.xyxy):
#                 x1, y1, x2, y2 = map(int, box[:4])
#                 class_id = int(result.boxes.cls[i])  # Assuming there's a 'cls' attribute for class ids
#                 class_label = model.names[class_id]
#                 cv2.rectangle(frame_resized, (x1, y1), (x2, y2), (255, 0, 0), 2)
#                 # Put class label above the bounding box
#                 cv2.putText(frame_resized, class_label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

#         # Convert image to JPEG format for display
#         _, jpeg = cv2.imencode('.jpeg', frame_resized)
#         image_widget.value = jpeg.tobytes()

#         # Allow the loop to be interrupted by stopping the kernel
#         try:
#             plt.pause(0.001)
#         except KeyboardInterrupt:
#             break

#     # When everything done, release the capture and close windows
#     cap.release()

# # Run the function to start capturing and displaying
# capture_and_display()